In [1]:
#Import libraries

#Data preprocessing
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

# label encoder,Standardize the data
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv(r"F:\Tutorials\Certified program in MACHINE LEARNING and ARTIFICIAL INTELLIGENCE\Intermediate assessment\train_v9rqX0R.csv")
test = pd.read_csv(r"F:\Tutorials\Certified program in MACHINE LEARNING and ARTIFICIAL INTELLIGENCE\Intermediate assessment\test_AbJTz2l.csv")


#### exploratory analysis 

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


In [4]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [5]:
train.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,7060.000000,8523.000000,8523.000000,8523.000000,8523.000000
mean,12.857645,0.066132,140.992782,1997.831867,2181.288914
std,4.643456,0.051598,62.275067,8.371760,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.773750,0.026989,93.826500,1987.000000,834.247400
50%,12.600000,0.053931,143.012800,1999.000000,1794.331000
75%,16.850000,0.094585,185.643700,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [6]:
X_train=train.drop(columns=['Item_Outlet_Sales'])
y_train=train['Item_Outlet_Sales']

### finding missing values 

In [7]:
X_train.isnull().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
dtype: int64

In [8]:
#train = train.drop(["Item_Identifier", "Outlet_Identifier"], axis=1)


In [9]:
X_train.drop(columns=['Outlet_Size'],inplace=True)

In [10]:
test.drop(columns=['Outlet_Size'],inplace=True)

In [11]:
#filling in missing values of item weight according to item identifier
X_train['Item_Weight'] = X_train['Item_Weight'].fillna(X_train.groupby('Item_Identifier')['Item_Weight'].transform('mean'))
missing_values =X_train['Item_Weight'].isnull()
missing_index=X_train.loc[missing_values,'Item_Weight']
missing_index

927    NaN
1922   NaN
4187   NaN
5022   NaN
Name: Item_Weight, dtype: float64

In [12]:
#filling in missing values of item weight according to item type
X_train['Item_Weight'] = X_train['Item_Weight'].fillna(X_train.groupby('Item_Type')['Item_Weight'].transform('mean'))

In [13]:
missing_values = X_train['Item_Weight'].isnull()
missing_values.isnull().sum()

0

In [14]:
#filling in missing values of item weight according to item identifier
test['Item_Weight'] = test['Item_Weight'].fillna(test.groupby('Item_Identifier')['Item_Weight'].transform('mean'))
missing_values = test['Item_Weight'].isnull()
missing_index=test.loc[missing_values,'Item_Weight']
missing_index

58     NaN
482    NaN
882    NaN
884    NaN
1203   NaN
1297   NaN
1300   NaN
1571   NaN
1741   NaN
2302   NaN
2600   NaN
3619   NaN
3734   NaN
3875   NaN
3909   NaN
4298   NaN
4496   NaN
5346   NaN
5609   NaN
5615   NaN
Name: Item_Weight, dtype: float64

In [15]:
#filling in missing values of item weight according to item type
test['Item_Weight'] = test['Item_Weight'].fillna(test.groupby('Item_Type')['Item_Weight'].transform('mean'))

In [16]:
X_train.isnull().sum()


Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

In [17]:
test.isnull().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Location_Type         0
Outlet_Type                  0
dtype: int64

#### encoding categorical variables 

In [18]:
categorical_features = X_train.select_dtypes(include='object')


In [19]:
for i in categorical_features.columns:
    print(i,'..........>',X_train[i].nunique())

Item_Identifier ..........> 1559
Item_Fat_Content ..........> 5
Item_Type ..........> 16
Outlet_Identifier ..........> 10
Outlet_Location_Type ..........> 3
Outlet_Type ..........> 4


In [20]:
categorical_features = test.select_dtypes(include='object')
for i in categorical_features.columns:
    print(i,'..........>',test[i].nunique())

Item_Identifier ..........> 1543
Item_Fat_Content ..........> 5
Item_Type ..........> 16
Outlet_Identifier ..........> 10
Outlet_Location_Type ..........> 3
Outlet_Type ..........> 4


In [21]:
X_train.drop(columns=['Item_Identifier'],inplace=True)
test.drop(columns=['Item_Identifier'],inplace=True)

In [22]:
# Label encoding 'Outlet_Location_Type'
label_encoder = LabelEncoder()
test['Outlet_Location_Type']= label_encoder.fit_transform(test['Outlet_Location_Type'])
X_train['Outlet_Location_Type']= label_encoder.fit_transform(X_train['Outlet_Location_Type'])
# Label encoding 'Item_Fat_Content'
label_encoder = LabelEncoder()
test['Item_Fat_Content']= label_encoder.fit_transform(test['Item_Fat_Content'])
X_train['Item_Fat_Content']= label_encoder.fit_transform(X_train['Item_Fat_Content'])


In [23]:
test= pd.get_dummies(test, columns=[ 'Item_Type', 'Outlet_Identifier', 'Outlet_Type'])

In [24]:
X_train=pd.get_dummies(X_train, columns=[ 'Item_Type', 'Outlet_Identifier', 'Outlet_Type'])

In [25]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Item_Weight                      8523 non-null   float64
 1   Item_Fat_Content                 8523 non-null   int32  
 2   Item_Visibility                  8523 non-null   float64
 3   Item_MRP                         8523 non-null   float64
 4   Outlet_Establishment_Year        8523 non-null   int64  
 5   Outlet_Location_Type             8523 non-null   int32  
 6   Item_Type_Baking Goods           8523 non-null   uint8  
 7   Item_Type_Breads                 8523 non-null   uint8  
 8   Item_Type_Breakfast              8523 non-null   uint8  
 9   Item_Type_Canned                 8523 non-null   uint8  
 10  Item_Type_Dairy                  8523 non-null   uint8  
 11  Item_Type_Frozen Foods           8523 non-null   uint8  
 12  Item_Type_Fruits and

In [26]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5681 entries, 0 to 5680
Data columns (total 36 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Item_Weight                      5681 non-null   float64
 1   Item_Fat_Content                 5681 non-null   int32  
 2   Item_Visibility                  5681 non-null   float64
 3   Item_MRP                         5681 non-null   float64
 4   Outlet_Establishment_Year        5681 non-null   int64  
 5   Outlet_Location_Type             5681 non-null   int32  
 6   Item_Type_Baking Goods           5681 non-null   uint8  
 7   Item_Type_Breads                 5681 non-null   uint8  
 8   Item_Type_Breakfast              5681 non-null   uint8  
 9   Item_Type_Canned                 5681 non-null   uint8  
 10  Item_Type_Dairy                  5681 non-null   uint8  
 11  Item_Type_Frozen Foods           5681 non-null   uint8  
 12  Item_Type_Fruits and

#### modelling SVR

In [27]:
def scale_std(X_train,X_test):
    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)
    return X_train_std,X_test_std

In [28]:
X_train_std,X_test_std=scale_std(X_train,test)

In [29]:
X_train_std=pd.DataFrame(X_train_std)

In [30]:
X_train_std

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,-0.769777,-0.572844,-0.970732,1.747454,0.139541,-1.369334,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,-0.349762,-0.349973,2.857362,-0.381529,0.726802,-0.349551,-0.351028
1,-1.497482,0.978092,-0.908111,-1.489023,1.334103,1.091569,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,-0.349762,-0.349973,-0.349973,-0.381529,-1.375891,2.860816,-0.351028
2,0.995659,-0.572844,-0.956917,0.010040,0.139541,-1.369334,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,-0.349762,-0.349973,2.857362,-0.381529,0.726802,-0.349551,-0.351028
3,1.361665,0.978092,-1.281758,0.660050,0.020085,1.091569,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,-0.349762,-0.349973,-0.349973,2.621031,-1.375891,-0.349551,-0.351028
4,-0.849437,-0.572844,-1.281758,-1.399220,-1.293934,1.091569,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,-0.349762,-0.349973,-0.349973,-0.381529,0.726802,-0.349551,-0.351028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,-1.294026,-0.572844,-0.181193,1.180783,-1.293934,1.091569,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,-0.349762,-0.349973,-0.349973,-0.381529,0.726802,-0.349551,-0.351028
8519,-0.967851,0.978092,-0.371154,-0.527301,0.497909,-0.138882,3.486083,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,2.859087,-0.349973,-0.349973,-0.381529,0.726802,-0.349551,-0.351028
8520,-0.489891,-0.572844,-0.599784,-0.897208,0.736822,-0.138882,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,2.857362,-0.349762,-0.349973,-0.349973,-0.381529,0.726802,-0.349551,-0.351028
8521,-1.219748,0.978092,1.532880,-0.607977,1.334103,1.091569,-0.286855,-0.174193,-0.114346,-0.287094,...,-0.256985,-0.351028,-0.349973,-0.349762,-0.349973,-0.349973,-0.381529,-1.375891,2.860816,-0.351028


In [31]:
y_train

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [32]:
### SVM
kf=KFold(n_splits=5)  # cross validation with Kfold
model= SVR()
model.fit(X_train_std, y_train)
ypred=model.predict(X_train_std)

In [33]:
rmse=mean_squared_error(ypred,y_train)
print('RMSE:',rmse)

RMSE: 2856858.3468429856


### Hyperparameter tuning 

In [34]:
parameters ={'C': [0.1, 1, 10, 100, 1000],
             'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
             'kernel': ['rbf']}

grid_search = GridSearchCV( SVR(), 
                            parameters, 
                            cv=5,
                            scoring='accuracy',n_jobs=-1)

In [35]:
grid_result= grid_search.fit(X_train_std,y_train)
print('Best Params: ', grid_result.best_params_)
print('Best Score: ', grid_result.best_score_)

KeyboardInterrupt: 

In [ ]:
### SVM with parameter optimization
model=SVR(C=100, gamma= 0.1,kernel='rbf')
model.fit(X_train_std, y_train)
ypred=model.predict(X_train_std)
err=accuracy_score(y_train, ypred)
score=cross_val_score(model,X_train,y_train,cv=kf).mean()
print(".......................................................................")
print(model)
print(".......................................................................")
print('{}% of the test samples are corrrectly classified'.format(err*100))
print('{}% of the test samples are corrrectly classified using K-Fold validation\n'.format(score*100))

In [36]:
ypred=model.predict(X_test_std)

In [39]:
test1 = pd.read_csv(r"F:\Tutorials\Certified program in MACHINE LEARNING and ARTIFICIAL INTELLIGENCE\Intermediate assessment\test_AbJTz2l.csv")
outputDataFrame=pd.DataFrame({'Item_Identifier':test1['Item_Identifier'],'Outlet_Identifier':test1['Outlet_Identifier'],'Item_Outlet_Sales':ypred})

outputDataFrame.to_csv("output_predicted.csv", index=False)